### Reddit Fake News Detector Project

This project is another personal project of mine.

In [ ]:
'''
#pip install praw 

NOTE: We install praw specifically using pip here because Conda version is
outdated. A seperate Conda environment was created to avoid package management issues.
'''

In [ ]:
import random #import these packages for the following praw program in the next cell
import socket
import sys

import time #import time to assign a counter to the incoming subreddit stream
import praw as praw #import praw package for streaming content from Reddit

In [ ]:
'''
Code taken from praw documentation to initiate program to obtain refresh token for Reddit API
authorisation. This is necessary to avoid using personal username and password for
authentication.
'''
def receive_connection():
    """Wait for and then return a connected socket..

    Opens a TCP connection on port 8080, and waits for a single client.

    """
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(("localhost", 8080))
    server.listen(1)
    client = server.accept()[0]
    server.close()
    return client


def send_message(client, message):
    """Send message to client and close the connection."""
    print(message)
    client.send(f"HTTP/1.1 200 OK\r\n\r\n{message}".encode("utf-8"))
    client.close()


def main():
    """Provide the program's entry point when directly executed."""
    print(
        "Go here while logged into the account you want to create a token for: "
        "https://www.reddit.com/prefs/apps/"
    )
    print(
        "Click the create an app button. Put something in the name field and select the"
        " script radio button."
    )
    print("Put http://localhost:8080 in the redirect uri field and click create app")
    client_id = input(
        "Enter the client ID, it's the line just under Personal use script at the top: "
    )
    client_secret = input("Enter the client secret, it's the line next to secret: ")
    commaScopes = input(
        "Now enter a comma separated list of scopes, or all for all tokens: "
    )

    if commaScopes.lower() == "all":
        scopes = ["*"]
    else:
        scopes = commaScopes.strip().split(",")

    reddit = praw.Reddit(
        client_id=client_id.strip(),
        client_secret=client_secret.strip(),
        redirect_uri="http://localhost:8080",
        user_agent="praw_refresh_token_example",
    )
    state = str(random.randint(0, 65000))
    url = reddit.auth.url(scopes, state, "permanent")
    print(f"Now open this url in your browser: {url}")
    sys.stdout.flush()

    client = receive_connection()
    data = client.recv(1024).decode("utf-8")
    param_tokens = data.split(" ", 2)[1].split("?", 1)[1].split("&")
    params = {
        key: value for (key, value) in [token.split("=") for token in param_tokens]
    }

    if state != params["state"]:
        send_message(
            client,
            f"State mismatch. Expected: {state} Received: {params['state']}",
        )
        return 1
    elif "error" in params:
        send_message(client, params["error"])
        return 1

    refresh_token = reddit.auth.authorize(params["code"])
    send_message(client, f"Refresh token: {refresh_token}")
    return 0


if __name__ == "__main__":
    sys.exit(main())
    

In [ ]:
reddit = praw.Reddit(client_id = "", 
                                   client_secret = "",
                                  user_agent = "",
                                  refresh_token = ""
                                  )

#Authentication credentials used above to get access to Reddit API
#Insert generated refresh token from the previous cell into the empty parameter

In [ ]:
print(reddit.user.me()) #check that the user login worked correctly

In [ ]:
subreddit = reddit.subreddit("ukpolitics") #Set the subreddit of choice to stream Reddit post submissions. 

In [ ]:
streamer = subreddit.stream.submissions()

In [ ]:
timeout = time.time() + 5

In [ ]:
#This is a defined function to stream titles from the subreddit
for submission in subreddit.stream.submissions():
    
    print(submission.title) 
    
return(submission)